<a href="https://colab.research.google.com/github/JacksonLeb/tensorflow_learning/blob/main/08_rnn_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#introduction to NLP fundamentals in Tf

##Getting info from natural language (sequences or speech)
##NLP can also be called seq2seq problems

###Checking for GPU


In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-ee4d1da0-c26a-898d-d244-d144f6a708b2)


In [2]:
##Get helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-07-26 21:28:40--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-26 21:28:40 (83.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
##Get a text dataset
###Using the Kaggle Intro to NLP dataset, samples of tweets labeled hazard or not
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-07-26 21:28:45--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.148.128, 142.250.128.128, 142.251.6.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.148.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2022-07-26 21:28:45 (52.7 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

##Visualizing a text dataset

In [6]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [8]:
train_df["text"][10]

'Three people died from the heat wave so far'

In [9]:
train_df["text"][100]

'.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad'

In [10]:
#shuffle trianing df
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [11]:
#test df
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [12]:
#examples of each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
#total num of samples
len(train_df), len(test_df)

(7613, 3263)

In [14]:
#visualizing random samples
import random
random_index = random.randint(0, len(train_df)-5) #create random index
for row in train_df_shuffled[["text", "target"]][random_index:random_index +5 ].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}")

Target: 1 (real disaster
Text:
I liked a @YouTube video http://t.co/43sXG9Z6xh TREMOR IS NO JOKE!! [TREMOR DLC] [FATALITIES/X-RAY]
Target: 1 (real disaster
Text:
@leedsrouge Love what you picked! We're playing WORTH IT by FIFTH HARM/KID INK because of you! Listen &amp; Vote: http://t.co/0wrATkA2jL
Target: 1 (real disaster
Text:
California meets drought-fueled fire season with extra crews.. Related Articles: http://t.co/rKDzB0TGC3
Target: 0 (not real disaster)
Text:
'Biggest lead hazard in New England history.' Yeah let's nail these guys. https://t.co/xzvmzQuS0x
Target: 0 (not real disaster)
Text:
Its like I never left. I just sunk to the background


In [15]:
#splittting data into training and validation
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=30)

In [16]:
#checking lengths of each data set
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [17]:
#check the first 10 sentences
train_sentences[:10]

array(["All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       "@DavidJordan88 @Stephanenny Except we don't know who started the riot or if it even makes sense to credit any particular individuals...",
       'Shadow boxing the apocalypse',
       "Maybe that's what happens when a tornado meets a volcano",
       '#Politics Democracy\x89Ûªs hatred for hate: \x89Û_ Dawabsha threaten to erode Israeli democracy. Homegrown terrorism ha...  http://t.co/q8n5Tn8WME',
       "@TomDean86 he's alright but Hazard/Willian ain't gonna be shifted easily.",
       '0-day bug in fully patched OS X comes under active exploit to hijack Macs http://t.co/sbGiRvQvzb',
       'Three Israeli soldiers wounded in West Bank terrorist attack via /r/worldnews http://t.co/su4ZVWADj7',
       'If it\x89Ûªs a war you came to see you will never see a waved white flag in front me.\nI can\x89Ûªt end up dead I wont be misled.',
     

In [18]:
val_sentences[:10]

array(['Blaze is my bro http://t.co/UdKeSJ01mL',
       'I was taught at school in the 1970s that piracy slavery and suicide-bombing were purely historical. No one then expected them to re-occur',
       'http://t.co/FueRk0gWui Twelve feared killed in Pakistani air ambulance helicopter crash http://t.co/Mv7GgGlmVc',
       "I'm a friggin wreck destiel sucks (read the vine description)\nhttps://t.co/MKX6Ux4OZt",
       'Dr. Bengston on #wildfire management: \x89ÛÏnumbers and size of fires areas affected and costs of fighting them all show upward trend.\x89Û\x9d #smem',
       '@ameenshaikh3 sir i just only wanted to make a point about @sureshpprabhu you made and said he is lying about bridge collapse.',
       'Wow! I just won this for free The Hobbit: Desolation of Smaug UV digital download code *GIN 9 http://t.co/MjFdCrjs8j #listia',
       'Photoset: hakogaku: ?åÊI am a kurd. i was born on a battlefield. raised on a battlefield. gunfire sirens... http://t.co/obp595W7tm',
       'Wash

## Converting text into numbers

#Tokenization and Embedding
##Tokenization: Each word/character is set to a number/one-hot encoding
##Embedding: Each word/character is set to a vector, with user-defined size, allowing for information on the relationship between tokens/words/characters. Additionally, can limit size and can be learned (treated as a weight matrix)


In [19]:
#Text Vectorization
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

#Use default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, #how many words in vocabulary (automaticaly add <OOV>)
                                    standardize="lower_and_strip_punctuation", #remove fluff,
                                    split="whitespace",
                                    ngrams=None, #create groups of n words
                                    output_mode="int", #how to map tokens to numbers
                                    output_sequence_length=None, #how long each batch of a setence should be
                                    pad_to_max_tokens=False)


In [20]:
#find average number of tokens (words) in the trianing tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [21]:
#set up text vectorization variables
max_vocab_length = 10000 # max num of words to have in vocabulary
max_length = 15 #max length of a tweet that our models sees

text_vectorizer = TextVectorization(max_tokens=max_vocab_length, #how many words in vocabulary (automaticaly add <OOV>)
                                    standardize="lower_and_strip_punctuation", #remove fluff,
                                    split="whitespace",
                                    ngrams=None, #create groups of n words
                                    output_mode="int", #how to map tokens to numbers
                                    output_sequence_length=max_length, #how long each batch of a setence should be
                                    pad_to_max_tokens=True)

In [22]:
#adapting text_vectorizer to training data
text_vectorizer.adapt(train_sentences) #adapt to training data, very important

In [23]:
train_sentences[:10]

array(["All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       "@DavidJordan88 @Stephanenny Except we don't know who started the riot or if it even makes sense to credit any particular individuals...",
       'Shadow boxing the apocalypse',
       "Maybe that's what happens when a tornado meets a volcano",
       '#Politics Democracy\x89Ûªs hatred for hate: \x89Û_ Dawabsha threaten to erode Israeli democracy. Homegrown terrorism ha...  http://t.co/q8n5Tn8WME',
       "@TomDean86 he's alright but Hazard/Willian ain't gonna be shifted easily.",
       '0-day bug in fully patched OS X comes under active exploit to hijack Macs http://t.co/sbGiRvQvzb',
       'Three Israeli soldiers wounded in West Bank terrorist attack via /r/worldnews http://t.co/su4ZVWADj7',
       'If it\x89Ûªs a war you came to see you will never see a waved white flag in front me.\nI can\x89Ûªt end up dead I wont be misled.',
     

In [24]:
#Create sample setnces and tokenize it
sample_sentence = "A tornado destroyed my house"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  3, 595, 346,  13, 311,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [25]:
#Choose a random sentence from the triaing dataset and vectorize it
random_sentence = random.choice(train_sentences)
print(f"original sentence: {random_sentence}\n vectorized sentence: {text_vectorizer([random_sentence])}")

original sentence: Panoramic Resources cuts jobs after seismic event http://t.co/mUwmfJGzYh
 vectorized sentence: [[9989 4712    1  809   40  648 1418    1    0    0    0    0    0    0
     0]]


In [26]:
#number of unique words found in train_sentences
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Num of words: {len(words_in_vocab)}")
top_5_words, bottom_5_words

Num of words: 10000


(['', '[UNK]', 'the', 'a', 'in'],
 ['panda', 'panama', 'pampalmater', 'palmoil', 'palmer'])

### Creating an embedding using an embedding layer


In [27]:
#using tensorflow.keras.layeres.Embedding
##Parameters: input_dim, output_dim, input_length
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)
embedding

In [28]:
#get a random sentence
random_sentence = random.choice(train_sentences)
print(f"random sentence: {random_sentence}")
print(f"embedded:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

random sentence: I can probably skip on these basic life maintenance things for a few days. (cut to burning buildings people screaming in the streets)
embedded:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.0368443 , -0.01600734, -0.00937449, ...,  0.00028076,
          0.02494489, -0.0241089 ],
        [-0.00776733, -0.04339819, -0.03612364, ..., -0.04791777,
         -0.03196694, -0.00602273],
        [ 0.04906208, -0.01253027, -0.03786515, ..., -0.02558098,
          0.01285389,  0.04938587],
        ...,
        [-0.01024687, -0.04091535, -0.01002855, ..., -0.03769626,
         -0.04486383,  0.0183211 ],
        [ 0.01621013, -0.01467055, -0.03629965, ..., -0.0084193 ,
          0.02920978,  0.02202323],
        [-0.01504396, -0.0143173 ,  0.03942715, ...,  0.04735645,
          0.00484189,  0.02596141]]], dtype=float32)>

In [29]:
sample_embed[0][0], sample_embed[0][0].shape, 

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-3.68443020e-02, -1.60073414e-02, -9.37448815e-03, -4.38351631e-02,
        -3.52601409e-02, -4.29368615e-02, -4.55364957e-02, -3.85195836e-02,
        -1.31833926e-02, -4.99206781e-03, -2.37170104e-02, -1.25099532e-02,
         4.05368917e-02,  2.82929279e-02, -3.20169218e-02, -4.94330041e-02,
         4.05974053e-02,  2.29937471e-02,  4.09019478e-02, -1.34213455e-02,
         3.54158022e-02,  1.92492492e-02,  2.48801745e-02,  1.53619535e-02,
        -2.23294385e-02,  1.65385865e-02, -1.87413581e-02,  2.30629109e-02,
         2.86393203e-02, -3.75800356e-02,  7.00627640e-03,  3.09771411e-02,
         4.44393642e-02,  8.48401338e-04,  3.45602371e-02, -2.49132514e-02,
         2.38928683e-02,  4.29136418e-02, -1.06425881e-02, -5.12401015e-03,
        -3.70829105e-02, -4.04690281e-02, -7.43650272e-03,  4.77502830e-02,
        -2.17554569e-02,  4.47491519e-02, -3.36519107e-02, -2.79536601e-02,
         3.92803289e-02,  3.33852060e-0

##Modelling a text dataset (running series of experiments)
###1. Naive bayes with Tf-IDF encoder (BASELINE)
###2. Feed-forward neural network 
###3. LSTM
###4. GRU
###5. Bidirectiona;-LSTM
###6. 1D CNN
###7. TFHub pretrianed feature extractor
###8. TFHub pretrained with 10% of trained datam



In [30]:
###Model 0: Getting a baseline - Skleanrs multinomial naive bayes with tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), #convert words to numbers
                    ("clf", MultinomialNB()) #model the text

])

#fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [31]:
#Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)

In [32]:
baseline_score

0.8044619422572179

In [33]:
print(f"Baseline Score: {baseline_score*100:.2f}%")

Baseline Score: 80.45%


In [34]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:30]

array([0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 1])

In [35]:
### creating an evaluation function for our modeling experiements
#Accuracy, Precision, Recall and f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculate model accuracy, preicision, recall and f1-score
  """
  #calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  
  #precision, recall and f1-score(using weighted average)
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  
  model_results = {
      "accuracy": model_accuracy,
      "precision": model_precision,
      "recall": model_recall,
      "f1": model_f1
  }

  return model_results

In [36]:
baseline_results = calculate_results(val_labels, baseline_preds)
baseline_results

{'accuracy': 80.4461942257218,
 'f1': 0.7993754401853582,
 'precision': 0.8113250534023555,
 'recall': 0.8044619422572179}

In [37]:
### Model 1: Feed Forward NN

# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"


In [38]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [39]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [40]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [41]:
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20220726-212853
Epoch 1/5
215/215 [==============================] - 8s 6ms/step - loss: 0.6142 - accuracy: 0.6878 - val_loss: 0.5170 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4433 - accuracy: 0.8218 - val_loss: 0.4423 - val_accuracy: 0.7953
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3486 - accuracy: 0.8584 - val_loss: 0.4340 - val_accuracy: 0.8150
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2860 - accuracy: 0.8883 - val_loss: 0.4361 - val_accuracy: 0.8176
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2389 - accuracy: 0.9092 - val_loss: 0.4601 - val_accuracy: 0.7940


In [42]:
#check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4601 - accuracy: 0.7940


[0.46011093258857727, 0.7939632534980774]

In [43]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs

array([[5.68793118e-02],
       [6.90988421e-01],
       [9.98193562e-01],
       [1.18522897e-01],
       [9.78440881e-01],
       [6.46007210e-02],
       [7.38340383e-03],
       [1.97376102e-01],
       [9.96243000e-01],
       [9.82323229e-01],
       [2.94226743e-02],
       [2.17307672e-01],
       [9.99538898e-01],
       [6.42023087e-02],
       [4.36827801e-02],
       [5.33563852e-01],
       [9.98392642e-01],
       [2.65316844e-01],
       [6.33162022e-01],
       [4.78869826e-01],
       [1.19367152e-01],
       [3.29248637e-01],
       [9.31547105e-01],
       [5.82983077e-01],
       [3.72290790e-01],
       [9.98727739e-01],
       [2.31619760e-01],
       [3.02960090e-02],
       [9.69763875e-01],
       [9.99854684e-01],
       [9.99255598e-01],
       [1.71176910e-01],
       [4.50929821e-01],
       [3.24511006e-02],
       [7.75086880e-02],
       [9.69903827e-01],
       [7.07242917e-03],
       [3.84042948e-01],
       [3.93359780e-01],
       [5.69148995e-02],


In [44]:
model_1_pred_probs[0]

array([0.05687931], dtype=float32)

In [45]:
model_1_pred_probs.shape

(762, 1)

In [46]:
model_1_pred_probs[:10]

array([[0.05687931],
       [0.6909884 ],
       [0.99819356],
       [0.1185229 ],
       [0.9784409 ],
       [0.06460072],
       [0.0073834 ],
       [0.1973761 ],
       [0.996243  ],
       [0.9823232 ]], dtype=float32)

In [47]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1.,
       0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0.,
       0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1.,
       1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0.,
       0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1

In [48]:
#Calculate model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 79.39632545931758,
 'f1': 0.7936549291543362,
 'precision': 0.7934825934657364,
 'recall': 0.7939632545931758}

In [49]:
baseline_results = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [50]:
words_in_vocab

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'with',
 'it',
 'that',
 'at',
 'by',
 'this',
 'from',
 'be',
 'are',
 'was',
 'have',
 'like',
 'me',
 'as',
 'up',
 'just',
 'but',
 'so',
 'im',
 'not',
 'amp',
 'your',
 'out',
 'its',
 'all',
 'will',
 'after',
 'no',
 'fire',
 'has',
 'an',
 'when',
 'if',
 'get',
 'now',
 'we',
 'about',
 'new',
 'via',
 'more',
 'or',
 'people',
 'what',
 'news',
 'one',
 'how',
 'he',
 'they',
 'over',
 'dont',
 'been',
 'who',
 'into',
 'video',
 'were',
 'do',
 'can',
 '2',
 'us',
 'disaster',
 'emergency',
 'there',
 'police',
 'than',
 'would',
 'still',
 'his',
 'some',
 'her',
 'body',
 'burning',
 'storm',
 'crash',
 'back',
 'had',
 'suicide',
 'man',
 'why',
 'know',
 'got',
 'them',
 'buildings',
 'world',
 'time',
 'off',
 'see',
 'california',
 'first',
 'day',
 'rt',
 'bomb',
 'youtube',
 'cant',
 'love',
 'nuclear',
 'going',
 'their',
 'dead',
 'two',
 'train',
 'killed',
 'go'

In [51]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [52]:
#get weight matrix of embedding layer
#numerical representations of each token in our training data, learned for 5 epochs
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [53]:
embed_weights

array([[ 0.02067396, -0.03137651, -0.0256853 , ...,  0.0332809 ,
         0.02185918, -0.04110548],
       [ 0.00941567, -0.04390863, -0.01144137, ..., -0.01998647,
         0.03199439, -0.0354187 ],
       [ 0.04797686, -0.04616719,  0.04812387, ..., -0.04899676,
         0.04264632, -0.0176123 ],
       ...,
       [-0.04369865,  0.03549594, -0.11140639, ...,  0.03668834,
        -0.07150431, -0.05931189],
       [-0.07915282,  0.04015886,  0.00857119, ...,  0.07443297,
        -0.082226  , -0.07792288],
       [ 0.08639867, -0.03349399,  0.06666066, ..., -0.02923373,
         0.0288353 ,  0.07209289]], dtype=float32)

In [54]:
#projector tool
import io
out_v = io.open('vectors.tsv', 'w', encoding="utf-8")
out_m = io.open('metadata.tsv', 'w', encoding="utf-8")

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue

  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [55]:
#download files from colab and upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Recurrent Neural Network

##useful for sequence data - use representation of previous input to aid the representation of a later input

In [59]:
### Model 2: LSTM
###LSTM = Long Short Term Memory (one of the most popular RNN)
'''
 input(text) -> toeknize -> embed -> layers (RNNs/Dense) -> Output
'''

'\n input(text) -> toeknize -> embed -> layers (RNNs/Dense) -> Output\n'

In [66]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
#x = layers.LSTM(64, return_sequences=True)(x)
#print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
#x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 64)


In [67]:
#Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_7 (LSTM)               (None, 64)                49408     
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [68]:
#compile
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [69]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220726-215006
Epoch 1/5
215/215 [==============================] - 9s 14ms/step - loss: 0.2185 - accuracy: 0.9241 - val_loss: 0.5306 - val_accuracy: 0.7900
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1571 - accuracy: 0.9423 - val_loss: 0.5458 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1325 - accuracy: 0.9527 - val_loss: 0.7055 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1028 - accuracy: 0.9596 - val_loss: 0.7279 - val_accuracy: 0.7559
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0874 - accuracy: 0.9650 - val_loss: 1.0608 - val_accuracy: 0.7441


In [70]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[2.4576774e-03],
       [9.8581481e-01],
       [9.9982733e-01],
       [1.1489737e-02],
       [9.9982470e-01],
       [6.0913898e-03],
       [5.6041317e-04],
       [7.9026166e-03],
       [9.9985230e-01],
       [9.9973398e-01]], dtype=float32)

In [72]:
#convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 0., 0., 0., 1., 1.], dtype=float32)>

In [73]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)

In [74]:
model_2_results

{'accuracy': 74.40944881889764,
 'f1': 0.7453630731452584,
 'precision': 0.7501311284355908,
 'recall': 0.7440944881889764}

In [76]:
#Model 3: GRU
#GRU = Gated recurrent unit
##GRU is similar to LSTM, but has less parameters

In [99]:
from tensorflow.keras import layers

model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")


inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

'''
# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
# x = layers.GRU(64, return_sequences=True) # stacking recurrent cells requires return_sequences=True
x = layers.GRU(64)(x) 
# x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")
'''


'\n# Build an RNN using the GRU cell\ninputs = layers.Input(shape=(1,), dtype="string")\nx = text_vectorizer(inputs)\nx = model_3_embedding(x)\n# x = layers.GRU(64, return_sequences=True) # stacking recurrent cells requires return_sequences=True\nx = layers.GRU(64)(x) \n# x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell\noutputs = layers.Dense(1, activation="sigmoid")(x)\nmodel_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")\n'

In [100]:
#compile
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [101]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_5 (GRU)                 (None, 64)                37248     
                                                                 
 dense_17 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [94]:
# Fit model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20220726-221449
Epoch 1/5
215/215 [==============================] - 3s 8ms/step - loss: 0.1578 - accuracy: 0.9402 - val_loss: 0.8071 - val_accuracy: 0.7493
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0865 - accuracy: 0.9676 - val_loss: 0.8873 - val_accuracy: 0.7572
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0737 - accuracy: 0.9723 - val_loss: 0.8861 - val_accuracy: 0.7585
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0657 - accuracy: 0.9730 - val_loss: 1.0593 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0539 - accuracy: 0.9750 - val_loss: 1.1937 - val_accuracy: 0.7507


In [96]:
model_3_preds = tf.squeeze(tf.round(model_3.predict(val_sentences)))
model_3_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1.,
       0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0.,
       1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1

In [97]:
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)

In [98]:
model_3_results

{'accuracy': 75.06561679790026,
 'f1': 0.7513408330263004,
 'precision': 0.7526882947827785,
 'recall': 0.7506561679790026}

In [103]:
#Model 4: Bidirectional RNN
##goes left to right and right to left

In [108]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4")

In [109]:
model_4.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_6 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_21 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-tra

In [110]:
#compile
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [111]:
# Fit model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "Bidrectional")])

Saving TensorBoard log files to: model_logs/Bidrectional/20220726-231759
Epoch 1/5
215/215 [==============================] - 8s 15ms/step - loss: 0.1081 - accuracy: 0.9696 - val_loss: 0.9426 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0536 - accuracy: 0.9762 - val_loss: 1.2133 - val_accuracy: 0.7546
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0473 - accuracy: 0.9777 - val_loss: 1.2297 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0434 - accuracy: 0.9791 - val_loss: 1.2276 - val_accuracy: 0.7467
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0424 - accuracy: 0.9791 - val_loss: 1.2391 - val_accuracy: 0.7612


In [112]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs

array([[3.79128207e-04],
       [9.63261485e-01],
       [9.99965668e-01],
       [1.17093429e-03],
       [9.99939799e-01],
       [1.64047405e-02],
       [5.85848902e-05],
       [2.31915317e-03],
       [9.99234319e-01],
       [9.99945283e-01],
       [3.21053405e-04],
       [9.62224905e-04],
       [9.99990582e-01],
       [1.74180575e-04],
       [9.10055824e-05],
       [4.68805898e-03],
       [9.99980092e-01],
       [1.00257550e-03],
       [9.87244189e-01],
       [8.62708151e-01],
       [2.89962739e-02],
       [6.36417046e-03],
       [9.97761011e-01],
       [3.42260115e-02],
       [2.98240618e-03],
       [9.99972105e-01],
       [3.45131266e-04],
       [1.69665669e-04],
       [9.99883056e-01],
       [9.99981284e-01],
       [9.99980569e-01],
       [2.44566239e-03],
       [5.56201458e-01],
       [1.17868512e-04],
       [1.15425973e-04],
       [9.99910235e-01],
       [5.86337992e-05],
       [1.93684828e-02],
       [6.22876128e-03],
       [1.51212123e-04],


In [113]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))

In [114]:
model_4_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1.,
       0., 1., 1.], dtype=float32)>

In [116]:
#calc the results of biredcational model
model_4_results = calculate_results(val_labels, model_4_preds)

In [117]:
model_4_results

{'accuracy': 76.11548556430446,
 'f1': 0.761057159511443,
 'precision': 0.7609700467283487,
 'recall': 0.7611548556430446}